In [7]:
import numpy as np
import matplotlib.pyplot as plt
from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler
from mne.decoding import SPoC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split as tts
import mne
from sklearn.model_selection import StratifiedKFold
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator,
                          cross_val_multiscore, LinearModel, get_coef)
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE 
import os
import os.path as op
from os import listdir
from os.path import isfile, join
from sklearn.preprocessing import StandardScaler

In [2]:
raw=mne.io.read_raw_eeglab('C:\\USB Drive\\NewEEG-200s\\SA20140205\\20140205_1230.set',preload=True)

Reading C:\USB Drive\NewEEG-200s\SA20140205\20140205_1230.fdt
Reading 0 ... 838231  =      0.000 ...  1637.170 secs...


In [3]:
events = mne.find_events(raw, min_duration=1/raw.info['sfreq'], shortest_event=1)

5974 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]


In [4]:
# event_id : this dictionary is created from information from Dave's dissertation page 27
concepts={1:'BABY',2:'BELL',3:'BIRD',4:'BURP',5:'DOG',6:'DRUM',
              7:'KNOCK',8:'LAUGH',9:'PHONE',10:'TRAIN',11:'WATER'}

event_id = {'trail_start':  31, 
            'left_button':  131, 
            'right_button': 132,
            'congruent':    133,
            'incongruent':  134,
            'correct':      129,
            'incorrect':    130,
            # targets
            't_baby' :        135,
            't_bell':         136,
            't_bird':         137,
            't_burp':         138,
            't_dog':          139,
            't_drum':         140,
            't_knock':        141,
            't_laugh':        142,
            't_phone':        143,
            't_train':        144,
            't_water':        145,
            #flankers
            'f_baby' :        155,
            'f_bell':         156,
            'f_bird':         157,
            'f_burp':         158,
            'f_dog':          159,
            'f_drum':         160,
            'f_knock':        161,
            'f_laugh':        162,
            'f_phone':        163,
            'f_train':        164,
            'f_water':        165,
}
# event codes 1 - 24 represent flanker and target stim codes. They are NOT explicity defined in dissertation.
for i in range(1,13):
    event_id["flanker_stim_{0}".format(i)] = i
    
for j in range(13, 25):
    event_id["target_stim_{0}".format(j)] = j

In [5]:
epoch = mne.Epochs(raw, events=events, event_id = event_id, preload = True, )

5974 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 5974 events and 359 original time points ...
0 bad epochs dropped


In [13]:
X=epoch.get_data();
y=epoch.events[:, 2] 

In [8]:
mnf= mne.decoding.UnsupervisedSpatialFilter(NMF(),average=False)

In [10]:
mnf_data= mnf.fit_transform(X)

ValueError: Negative values in data passed to NMF (input X)

In [11]:
print (X<0)

[[[ True False  True ...  True  True  True]
  [ True False False ... False False False]
  [False False False ...  True  True  True]
  ...
  [ True  True  True ... False False False]
  [ True  True  True ...  True  True  True]
  [False False False ... False False False]]

 [[False  True  True ...  True  True  True]
  [False  True  True ... False False False]
  [False False False ...  True  True  True]
  ...
  [ True  True  True ... False False False]
  [ True  True  True ...  True  True  True]
  [False False False ... False False False]]

 [[ True  True  True ...  True  True  True]
  [ True  True  True ... False False False]
  [False False False ...  True  True  True]
  ...
  [ True  True  True ... False False False]
  [ True  True  True ...  True  True  True]
  [False False False ... False False False]]

 ...

 [[ True  True  True ...  True False False]
  [ True  True False ... False False False]
  [ True  True False ...  True  True  True]
  ...
  [False False  True ... False  True  Tr

In [13]:
PAC= mne.decoding.UnsupervisedSpatialFilter(PCA(25),average=False)

In [ ]:
PCA_data= PAC.fit_transform(X)

In [8]:
scaler = StandardScaler()

In [9]:
 print(scaler.fit(X))

ValueError: Found array with dim 3. StandardScaler expected <= 2.

In [15]:
clf = make_pipeline(Vectorizer(),
                        KNN(n_neighbors=13))
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)
    
clf.fit(X_train, y_train)
    
score = clf.score(X_test, y_test)
print (score)

MemoryError: 